<a href="https://colab.research.google.com/github/Victor-Andrade/Recommendation-System/blob/main/Mini_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Sistema de Recomendação
##Discentes: Maria Virgínia e Victor Hugo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import random
import numpy as np

# Caminhos dos arquivos
ratings_path = '/content/drive/My Drive/recomendacao de filmes/ml-32m/ratings.csv'
movies_path = '/content/drive/My Drive/recomendacao de filmes/ml-32m/movies.csv'

# Ler o arquivo de filmes
movies = pd.read_csv(movies_path)

# Definir o tamanho dos chunks e fração para amostragem
chunk_size = 50000  # Ajuste este valor conforme necessário
frac = 1  # Fração de dados a serem amostrados o.5 para obter 50% do dataset total

# Lista para armazenar os pedaços amostrados
sampled_chunks = []

# Ler o arquivo de ratings em pedaços e amostrar
for chunk in pd.read_csv(ratings_path, chunksize=chunk_size):
    # Fazer o merge do pedaço atual com os filmes
    chunk = pd.merge(chunk, movies, on='movieId')
    # Amostrar pedaço atual
    sampled_chunk = chunk.sample(frac=frac, random_state=42)
    sampled_chunks.append(sampled_chunk)

# Concatenar todos os pedaços amostrados em um único DataFrame
data_sample = pd.concat(sampled_chunks, axis=0)

# Dividir os dados em conjuntos de treino e teste
train_data, test_data = train_test_split(data_sample, test_size=0.5, random_state=42)

# Criar um mapeamento de IDs de usuários e títulos de filmes para índices numéricos
user_mapper = {user_id: i for i, user_id in enumerate(train_data['userId'].unique())}
movie_mapper = {movie_id: i for i, movie_id in enumerate(train_data['movieId'].unique())}

# Mapeando IDs para índices
user_index = train_data['userId'].map(user_mapper)
movie_index = train_data['movieId'].map(movie_mapper)

# Criar a matriz esparsa do conjunto de treino
user_movie_matrix_sparse = csr_matrix((train_data['rating'], (user_index, movie_index)))


In [18]:
# Função para calcular similaridade para um usuário específico sob demanda
def calculate_similarity_for_user(user_id, user_movie_matrix_sparse):
    # Índice do usuário para quem estamos calculando a similaridade
    user_idx = user_mapper.get(user_id)

    if user_idx is None:
        return None  # Usuário não está no conjunto de treino

    # Similaridade do usuário alvo com todos os outros usuários
    user_vector = user_movie_matrix_sparse[user_idx]
    similarity_scores = cosine_similarity(user_vector, user_movie_matrix_sparse).flatten()

    # Criar um DataFrame para similaridades com índice de userId original
    similarity_df = pd.DataFrame(data=similarity_scores, index=list(user_mapper.keys()), columns=['similarity'])

    # Ordenar pela similaridade, do maior para o menor
    return similarity_df.sort_values('similarity', ascending=False)

# Função para recomendar filmes baseada na similaridade sob demanda
def recommend_movies(user_id, num_recommendations):
    similar_users = calculate_similarity_for_user(user_id, user_movie_matrix_sparse)
    if similar_users is None:
        return []  # Usuário não está no conjunto de treino

    recommendations = []

    # Iterar sobre usuários similares para encontrar recomendações
    for similar_user in similar_users.index[1:]:  # Pular o primeiro, que é o próprio usuário
        similar_user_ratings = train_data[train_data['userId'] == similar_user]
        recommended_movies = similar_user_ratings['title']
        recommendations.extend(recommended_movies)

        if len(recommendations) >= num_recommendations:
            break

    return random.sample(recommendations, min(num_recommendations, len(recommendations)))

user_id = user_index.iloc[5]  # Escolher um user_id válido da lista de IDs disponíveis
num_recommendations = 5
recommended_movies = recommend_movies(user_id, num_recommendations)
print("Recomendações para o usuário", user_id, ":", recommended_movies)

Recomendações para o usuário 5 : ['Outbreak (1995)', 'Jurassic Park (1993)', 'Cliffhanger (1993)', 'Fugitive, The (1993)', 'Batman (1989)']


In [19]:
def calculate_precision(user_id, recommended_movies, ratings_threshold=4.0):
    # Obter os filmes que o usuário avaliou e que são considerados relevantes
    user_ratings = train_data[train_data['userId'] == user_id]
    relevant_movies = user_ratings[user_ratings['rating'] >= ratings_threshold]['title'].tolist()

    # Contar quantos dos filmes recomendados são relevantes
    relevant_recommendations = [movie for movie in recommended_movies if movie in relevant_movies]

    # Calcular a precisão (número de recomendações relevantes / total de recomendações)
    precision = len(relevant_recommendations) / len(recommended_movies) if recommended_movies else 0
    return precision

# Exemplo de uso
precision = calculate_precision(user_id, recommended_movies)
print(f"Precisão para o usuário {user_id}: {precision:.4f}")


Precisão para o usuário 5: 0.6000


Avaliar usuários com mais ratings
